In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
DATADIR = "/scratch2/hugo/ERA5"
CLIMSTOR = "/mnt/climstor/ecmwf/era5/raw"
DATERANGEPL = pd.date_range("19590101", "20211231")
DATERANGEML = pd.date_range("19770101", "20211231")

In [ ]:
import cdsapi

c = cdsapi.Client()
for year in [2022]:
    if os.path.isfile(f'{DATADIR}/Geopotential/raw/{year}.nc'):
        continue
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': 'geopotential',
            'pressure_level': [
                '500',
            ],
            'year': str(year),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '06:00', '12:00',
                '18:00',
            ],
            'grid': '0.5/0.5',
            'format': 'netcdf',
        },
        f'{DATADIR}/Geopotential/raw/{year}.nc')

In [ ]:
c = cdsapi.Client()
for year in np.unique(DATERANGEPL.year)[:]:
    c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'variable': [
                'u_component_of_wind',
                'v_component_of_wind',
            ],
            'pressure_level': [
                '300'
            ],
            'year': str(year),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '06:00', '12:00',
                '18:00',
            ],
            'grid': '0.5/0.5',
            'format': 'netcdf',
        },
        f'{DATADIR}/Wind/{year}.nc')

In [ ]:
import cdsapi
import shutil

c = cdsapi.Client()
months = [str(i) for i in range(1, 13)]
years = [str(i) for i in range(1997, 2023)]
varname = "geopotential"
grid = "1.0/1.0"
plevel = "500"
for year in years:
    for month in months:
        if (year, month) in [("1997", f"{i}") for i in range(1, 12)]:
            continue 
        print("year", year, "month", month)
        result = c.service(
            "tool.toolbox.orchestrator.workflow",
            
            params={
                "realm": "user-apps",
                "project": 'app-c3s-daily-era5-statistics',
                "version": "master",
                "kwargs": {
                    "dataset": "reanalysis-era5-pressure-levels",
                    "product_type": "reanalysis",
                    "variable": varname,
                    "statistic": "daily_mean",
                    "year": year,
                    "month" : month,
                    "time_zone": "UTC+00:0",
                    "frequency": "1-hourly",
                    "grid": grid,
                    "area": {"lat": [15, 80], "lon": [-180, 179]},
                    "pressure_level": plevel,
                },
                "workflow_name": "application"
            }
        )
        c.download(result)
        
        filename = f"{DATADIR}/{varname}_{plevel}hPa_{year}{month}_{grid.split('/')[0]}deg.nc"
        shutil.move(result[0]["location"].split("/")[-1], filename)

In [ ]:
from ecmwfapi import ECMWFDataServer
server = ECMWFDataServer()
server.retrieve({
    "class": "e4",
    "dataset": "era40",
    "date": "1957-12-01/to/2002-03-31",
    "levelist": "700/775/850/925",
    "levtype": "pl",
    "param": "131.128/132.128",
    "step": "0",
    "stream": "oper",
    "time": "00:00:00/06:00:00/12:00:00/18:00:00",
    "type": "an",
    "target": "/scratch2/hugo/ERA40/UandV.nc",
})